In [4]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import pylab
import glob
import time
%matplotlib inline

/Users/Kim/Software/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## HDI Mosaic and Bias-Correction Procedure

### The following *incomplete* code is written as a starting point to achieve two goals: 

1) convert a multi-extension (4 amplifier) HDI image into a regular, single extension FITS image,  
and  
2) correct for the bias drift and subtract the bias signal from HDI images.

**Your tasks:**  
(1) Comment the code so it makes sense to you  
(2) Finish the code so it runs successfully on a test image  
(3) Apply the bias correction to a single night of data (both calibration frames and science frames) for your project  
(4) Save the newly-made mosaiced, bias-corrected images  
(5) Describe the how, what, and why of these reduction steps in your weekly research blog.

**What you will need:**
* A flatfield frame to test the mosaic part of the code (read_raw_fits)
* A set of flatfield frames to bias-correct



In [1]:
# How big should our mosaic image (all four quadrants in one image) be? 

# Initialize parameters for the final image size, below:

final_imx = 2056*2
final_imy = 2156*2

In [2]:
def read_raw_fits(im, newimx, newimy):
    """
    [add your own docstring here to describe the purpose of this function, filling out template below]

    Purpose: 
    
    Input:
    
    Returns:
    
    Example Usage:
    
    """
    im_padded = np.zeros([newimx, newimy])
    im_all = fits.open(im)
    im_header = im_all[0].header
    
    # im_padded[newimx/2:newimx, 0:newimy/2] places data in the upper left quadrant
    # im_all[4].data corresponds to the northeast quadrant of the image; needs to be inverted in Y
    
    # [(newimx/2):(newimx), newimy/2:(newimy)] places data in the upper right quadrant
    # im_all[3].data corresponds to the northwest quadrant of the image; needs to be inverted in X and Y
    
    # [0:newimx/2, 0:newimy/2] places data in lower left quadrant
    # im_all[2].data corresponds to the southeast quadrant of the image; needs no inversion
    
    # [0:newimx/2, newimy/2:newimy] places data in the lower right quadrant
    # im_all[1].data corresponds to the southwest quadrant of the image; needs to be inverted in X
    

    # northeast
    im_padded[newimx/2:newimx, 0:newimy/2] = np.flipud(im_all[4].data) # im_all[4].data[::-1]
    
    # northwest
    im_padded[(newimx/2):(newimx), newimy/2:(newimy)] = np.fliplr(np.flipud(im_all[3].data)) # im_all[3].data[::-1,::-1]
    
    # southeast
    im_padded[0:newimx/2, 0:newimy/2] = im_all[2].data
    
    # southwest
    im_padded[0:newimx/2, newimy/2:newimy] = np.fliplr(im_all[1].data)

    return im_padded, im_header
        

### In the cell below, try out the read_raw_fits function on a single test flatfield image.

In [5]:
# complete this:

test_mosaic_flat, test_mosaic_header = read_raw_fits('./sampleflats/iflats/c8131t0004f00.fits', final_imx, final_imy)

### Now save the file you've just read as a new FITS file. Call it *test_mosaic.fits*:

In [6]:
# complete this: your code to save the new file here

fits.writeto('test_mosaic.fits', test_mosaic_flat, test_mosaic_header, overwrite=True)

### Test the before/after FITS files by opening them in ds9. How did it work?

(Open the old one with File -> Open As -> Mosaic IRAF; the new one should open like a regular FITS file.)

In [7]:
# Note to KWD: Remove this cell!

# Make a master bias frame. First, create a list of the raw bias frames using glob. Complete this:

biases = glob.glob('./samplebias/c8130t0733b00.fits')

def create_master_bias(biaslist, newimx, newimy):
    """
    [add your own docstring here to describe the purpose of this function, filling out template below]

    Purpose: 
    
    Input:
    
    Returns:
    
    Example Usage:
    """
    
    im = fits.open(biaslist[0])
    
    biasim_h = im[0].header

    bias_cube = np.zeros([newimx, newimy, len(biaslist)])

    for ii in range(0, len(biaslist)):
        im, tmp = read_raw_fits(biaslist[ii], newimx, newimy)
        bias_cube[:,:,ii] = im
    
    median_bias = np.median(bias_cube, axis=2)
    return median_bias, biasim_h

## (to remove?) In the cell below, make the master bias and save it.

In [8]:
# Note to KWD: Remove this cell!

# make a note of the time
t0 = time.time()

# complete this:

master_bias, biasim_h = create_master_bias(biases, final_imx, final_imy)

# save the test bias
fits.writeto('test_bias.fits', master_bias, biasim_h, overwrite=True)

# make a note of the time
t1 = time.time()

# determine how long it took:
total = t1-t0

print("Total time to run: ", total, "sec")

('Total time to run: ', 4.164528846740723, 'sec')


## old bias-correction function

In [9]:
def bias_correct(fits_im, medbias, newimx, newimy):
    """
    Purpose: 
    For a given FITS image (fits_im), and a master bias frame (medbias), read in fits_im and measure the 
    overscan values for each quadrant, and measure the bias values for each quadrant in the master bias (medbias).
    Then, apply offsets to the master bias quadrants to make them match the overscan, in order to be able to
    subtract the appropriate bias level from each quadrant in the actual FITS image of interest.
        
    Input: 
    fits_im - raw science (or flat) frame to be corrected
    medbias - generic master bias constructed from a set of raw biases
    newimx - dimensions in x-axis 
    newimy - dimensions in y-axis 
    
    Returns: 
    corr_image - a bias-corrected version of the input science (or flat) image
    scale_xx - scale factor between the overscan level and bias quadrant level, per quadrant. xx can stand for 
                upper left (ul), upper right (ur), lower left (ll), and lower right (lr).
                
    Example Usage: 
    
    
    """
    # First, make your input science (or flat) image into a mosaic:
    imframe, imframe_hdr = read_raw_fits(fits_im, newimx, newimy)
    
    #[newimx/2:newimx, 0:newimy/2] => upper left quadrant
    overscan_ul = np.median(imframe[newimx/2:newimx, (newimy/2. - 108):newimy/2]) 
    bias_ul = np.median(medbias[newimx/2:newimx, 0:newimy/2]) # use this one
    bias_ul = np.median(medbias[newimx/2:newimx, (newimy/2. - 108):newimy/2]) 
    scale_ul = overscan_ul/bias_ul
    diff_ul = overscan_ul - bias_ul

    # [(newimx/2):(newimx), newimy/2:(newimy)] => upper right quadrant
    overscan_ur = np.median(imframe[(newimx/2):(newimx), newimy/2:(newimy/2 + 108)])
    bias_ur = np.median(medbias[(newimx/2):(newimx), newimy/2:(newimy)]) # use this one
    bias_ur = np.median(medbias[(newimx/2):(newimx), newimy/2:(newimy/2 + 108)])
    scale_ur = overscan_ur/bias_ur
    diff_ur = overscan_ur - bias_ur

    # [0:newimx/2, 0:newimy/2] => lower left quadrant
    overscan_ll = np.median(imframe[0:newimx/2, (newimy/2-108):newimy/2])
    bias_ll = np.median(medbias[0:newimx/2, 0:newimy/2]) # use this one
    bias_ll = np.median(medbias[0:newimx/2, (newimy/2-108):newimy/2])
    scale_ll = overscan_ll/bias_ll
    diff_ll = overscan_ll - bias_ll
    
    # [0:newimx/2, newimy/2:newimy] => lower right quadrant
    overscan_lr = np.median(imframe[0:newimx/2, (newimy/2):(newimy/2)+108])
    bias_lr = np.median(medbias[0:newimx/2, newimy/2:newimy]) # use this one
    bias_lr = np.median(medbias[0:newimx/2, (newimy/2):(newimy/2)+108])
    scale_lr = overscan_lr/bias_lr
    diff_lr = overscan_lr - bias_lr
    

    # Now re-scale each part of the master bias to match the overscan median, by first making a new empty image

    corr_bias = np.zeros([newimx, newimy])
    
    
    # Now apply the scaling factor to each quadrant in turn:

#     # ul
#     corr_bias[newimx/2:newimx, 0:newimy/2] = medbias[newimx/2:newimx, 0:newimy/2]*scale_ul 

#     # ur
#     corr_bias[(newimx/2):(newimx), newimy/2:(newimy)] = medbias[(newimx/2):(newimx), newimy/2:(newimy)]*scale_ur 

#     # ll
#     corr_bias[0:newimx/2, 0:newimy/2] = medbias[0:newimx/2, 0:newimy/2]*scale_ll 

#     # lr
#     corr_bias[0:newimx/2, newimy/2:newimy] = medbias[0:newimx/2, newimy/2:newimy]*scale_lr 
    
    
    # try it with diff
    
        # ul
    corr_bias[newimx/2:newimx, 0:newimy/2] = medbias[newimx/2:newimx, 0:newimy/2] + diff_ul
    

    # ur
    corr_bias[(newimx/2):(newimx), newimy/2:(newimy)] = medbias[(newimx/2):(newimx), newimy/2:(newimy)] + diff_ur
    

    # ll
    corr_bias[0:newimx/2, 0:newimy/2] = medbias[0:newimx/2, 0:newimy/2] + diff_ll
    

    # lr
    corr_bias[0:newimx/2, newimy/2:newimy] = medbias[0:newimx/2, newimy/2:newimy] + diff_lr
    
    

    
    # Lastly, subtract your newly-scaled bias from the mosaic image:
    
    bias_corr_image = imframe - corr_bias
    
    return bias_corr_image, corr_bias, imframe_hdr, scale_lr, scale_ll, scale_ur, scale_ul


## actual bias-correction function

In [10]:
def overscan_only_bias_correct(fits_im, newimx, newimy):
    """
    Purpose: 
    For a given FITS image (fits_im), read in imframe and measure the mean overscan values for each of its quadrants.
    Then, apply the offsets to each quadrant -- that is, subtract the appropriate bias level from each quadrant 
    in the FITS image of interest.
        
    Input: 
    fits_im - raw science (or flat) frame to be corrected
    newimx - dimensions in x-axis 
    newimy - dimensions in y-axis 
    
    Returns: 
    corr_image - a bias-corrected version of the input science (or flat) image
    im_hdr - header of the input science (or flat) image
                
    Example Usage: 
    new_corrected_im, corrected_im_header = overscan_only_bias_correct('image.fits', x_image_size, y_image_size)
    """
    
    # First, make your input science (or flat) image into a mosaic:
    imframe, imframe_hdr = read_raw_fits(fits_im, newimx, newimy)

    
    # Measure the overscans for each quadrant: 
    
    #[newimx/2:newimx, 0:newimy/2] => upper left quadrant
    overscan_ul = np.median(imframe[newimx/2:newimx, (newimy/2 - 108):newimy/2]) 
    
    # [(newimx/2):(newimx), newimy/2:(newimy)] => upper right quadrant
    overscan_ur = np.median(imframe[(newimx/2):(newimx), newimy/2:(newimy/2 + 108)])
    
    # [0:newimx/2, 0:newimy/2] => lower left quadrant
    overscan_ll = np.median(imframe[0:newimx/2, (newimy/2-108):newimy/2])

    # [0:newimx/2, newimy/2:newimy] => lower right quadrant
    overscan_lr = np.median(imframe[0:newimx/2, (newimy/2):(newimy/2)+108])

    

    # Now make a new empty image to place each subtracted quadrant
    corr_im = np.zeros([newimx, newimy])

    # ul
    corr_im[newimx/2:newimx, 0:newimy/2] = imframe[newimx/2:newimx, 0:newimy/2]- overscan_ul

    # ur
    corr_im[(newimx/2):(newimx), newimy/2:(newimy)] = imframe[(newimx/2):(newimx), newimy/2:(newimy)] - overscan_ur

    # ll
    corr_im[0:newimx/2, 0:newimy/2] = imframe[0:newimx/2, 0:newimy/2] - overscan_ll

    # lr
    corr_im[0:newimx/2, newimy/2:newimy] = imframe[0:newimx/2, newimy/2:newimy] - overscan_lr
    
    return corr_im, imframe_hdr


In [17]:
bias_corr_image, corr_bias, hdr1, scale_lr, scale_ll, scale_ur, scale_ul = bias_correct('./sampleflats/vflats/c8130t0742f00.fits', master_bias, final_imx, final_imy)

/Users/Kim/Software/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Kim/Software/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [18]:
bias_corr_image_overscan, hdr2 = overscan_only_bias_correct('./sampleflats/vflats/c8130t0742f00.fits', final_imx, final_imy)

In [19]:
# Write the file:
fits.writeto('corr_flat_overscan_only_V.fits', bias_corr_image_overscan, hdr, overwrite=True)

In [20]:
fits.writeto('corr_flat_V.fits', bias_corr_image, hdr, overwrite=True)

In [21]:
diff = bias_corr_image_overscan - bias_corr_image

In [22]:
np.median(diff)

4.0

In [23]:
np.std(diff)

8.2472237424615358

In [24]:
fits.writeto('diff_flat_V.fits', diff, hdr1, overwrite=True)